<a href="https://colab.research.google.com/github/safakatakancelik/portfolio-public/blob/main/notebooks/data/datasets/Superstore/data_prepared_with_tableau_prep/data_prep/Version_Oct_27_Product_ID_Conflict_Investigation_and_New_ID_Assignments_(Superstore_Dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-cpu
!pip install sentence-transformers

In [14]:
import pandas as pd
import numpy as np

In [15]:
# This is a data I prepared with Tableau Prep by joining the components of the Superstore dataset files that comes default with Tableau Prep
df = pd.read_csv("https://raw.githubusercontent.com/safakatakancelik/portfolio-public/refs/heads/main/notebooks/data/datasets/Superstore/data_prepared_with_tableau_prep/data_prep/output_all.csv")
df.head()

,Is Duplicate Row?,Days to Ship,Table Names,File Paths,Category,City,Country,Customer ID,Customer Name,Order Date,...,Discount,Product Name,Return Reason,Returned?,Return Notes,Return Approver,Quota-2015,Quota-2016,Quota-2017,Quota-2018
0,Unique,4,Orders_Central.csv,NaN,Furniture,Houston,United States,SF-20965,Sylvia Foulston,9/17/2016 12:00:00 AM,...,0.6,Howard Miller Distant Time Traveler Alarm Clock,NaN,No,NaN,NaN,100000,105000,120000,145000
1,Unique,6,Orders_Central.csv,NaN,Furniture,Lincoln Park,United States,DM-12955,Dario Medina,9/21/2016 12:00:00 AM,...,0.0,O'Sullivan 2-Shelf Heavy-Duty Bookcases,NaN,No,NaN,NaN,100000,105000,120000,145000
2,Unique,6,Orders_Central.csv,NaN,Office Supplies,Tulsa,United States,RB-19465,Rick Bensley,9/7/2015 12:00:00 AM,...,0.0,Avery Recycled Flexi-View Covers for Binding S...,NaN,No,NaN,NaN,100000,105000,120000,145000
3,Unique,3,Orders_East.xlsx/Orders_East,NaN,Office Supplies,Philadelphia,United States,ON-18715,Odella Nelson,9/24/2016 12:00:00 AM,...,0.2,Newell 320,NaN,No,NaN,NaN,125000,150000,175000,200000
4,Unique,2,Orders_West.csv,NaN,Office Supplies,Pasadena,United States,JF-15355,Jay Fein,4/23/2018 12:00:00 AM,...,0.2,GBC Durable Plastic Covers,NaN,No,NaN,NaN,115000,200000,225000,300000


#### Cleaning Product ID and Product Name Conflict
---

I did find product names sharing the same product IDs and run similarity search on them using 3 different models:
- TF-IDF KNN Cosine similarity
- TF-IDF FAISS
- Sentence Transformers FAISS


Based on these results all seem as if separate entities, I continue with this assumption and assign new unique product IDs for analysis purposes.


Additionally, I noticed same product names linking to different product IDs, analyzed and kept them as they are.

---

#### Part 1: Same Product IDs Linked with Different Product Names

In [16]:
# Products with the different names
df_prods = df[["Row ID", "Product ID", "Product Name"]].copy()

name_counts = df_prods.groupby("Product ID")["Product Name"].nunique()

ids_with_multiple_names = name_counts[name_counts > 1].reset_index()

df_prods_filtered = df_prods[df_prods["Product ID"].isin(ids_with_multiple_names["Product ID"])][["Product ID", "Product Name"]].drop_duplicates()

df_prods_filtered.sort_values(by="Product ID")

,Product ID,Product Name
938,FUR-BO-10002213,"Sauder Forest Hills Library, Woodland Oak Finish"
1552,FUR-BO-10002213,DMI Eclipse Executive Suite Bookcases
1836,FUR-CH-10001146,"Global Value Mid-Back Manager's Chair, Gray"
2091,FUR-CH-10001146,"Global Task Chair, Black"
184,FUR-FU-10001473,DAX Wood Document Frame
...,...,...
120,TEC-PH-10002200,Samsung Galaxy Note 2
2120,TEC-PH-10002310,Plantronics Calisto P620-M USB Wireless Speake...
4534,TEC-PH-10002310,Panasonic KX T7731-B Digital phone
3594,TEC-PH-10004531,OtterBox Commuter Series Case - iPhone 5 & 5s


In [17]:
# Similarity check with TF-IDF and Cosine Similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from IPython.display import display

corpus = df_prods_filtered["Product Name"].tolist()

# Vectorize
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(corpus)


# Compute cosine similarity matrix
cosine_sim_matrix = cosine_similarity(X)

# Nearest Neighbors
nbrs = NearestNeighbors(n_neighbors=6, metric="cosine").fit(X)
nbrs.fit(X)


distances, indices = nbrs.kneighbors(X)

threshold = 0.05 # starting threshold


for threshold in np.arange(0.05, 1.0, 0.05):
    similar_items = []
    for i, (dist, idx) in enumerate(zip(distances, indices)):
        for d, j in zip(dist[1:], idx[1:]):  # Skip the first one (itself)
            if d < threshold:
                similar_items.append({
                    "Product Name": df_prods_filtered.iloc[i]['Product Name'],
                    "Similar Product Name": df_prods_filtered.iloc[j]['Product Name'],
                    "Distance": d,
                    "Threshold": threshold
                })
    similar_items_df = pd.DataFrame(similar_items)
    if not similar_items_df.empty:
        print(f"Threshold: {threshold}")
        display(similar_items_df)
        break

# Similar matches found with the threshold of 0.25, statistically insignificant

Threshold: 0.25


,Product Name,Similar Product Name,Distance,Threshold
0,"Howard Miller 13-3/4"" Diameter Brushed Chrome ...","Howard Miller 14-1/2"" Diameter Chrome Round Wa...",0.201372,0.25
1,"Howard Miller 13-3/4"" Diameter Brushed Chrome ...","Howard Miller 13"" Diameter Goldtone Round Wall...",0.232074,0.25
2,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...","Howard Miller 13-3/4"" Diameter Brushed Chrome ...",0.201372,0.25
3,"Executive Impressions 13"" Clairmont Wall Clock","Executive Impressions 13"" Chairman Wall Clock",0.239840,0.25
4,"Howard Miller 13"" Diameter Goldtone Round Wall...","Howard Miller 13-3/4"" Diameter Brushed Chrome ...",0.232074,0.25
5,"Executive Impressions 13"" Chairman Wall Clock","Executive Impressions 13"" Clairmont Wall Clock",0.239840,0.25


In [18]:
# FAISS with TF-IDF

import faiss

# Using the same vectors but casting to float32 for compatibility with FAISS
X_faiss = X.toarray().astype('float32')

index = faiss.IndexFlatL2(X_faiss.shape[1])  # L2 distance
index.add(X_faiss)

# Search for the nearest neighbors
k = 6  # number of nearest neighbors
distances, indices = index.search(X_faiss, k)


for threshold in np.arange(0.05, 1.0, 0.05):
    similar_items = []
    for i, (dist, idx) in enumerate(zip(distances, indices)):
        for d, j in zip(dist[1:], idx[1:]):  # Skip the first one (itself)
            if d < threshold:
                similar_items.append({
                    "Product Name": df_prods_filtered.iloc[i]['Product Name'],
                    "Similar Product Name": df_prods_filtered.iloc[j]['Product Name'],
                    "Distance": d,
                    "Threshold": threshold
                })
    similar_items_df = pd.DataFrame(similar_items)
    if not similar_items_df.empty:
        print(f"Threshold: {threshold}")
        display(similar_items_df)
        break

  # Similar matches found with the threshold of 0.45, statistically insignificant

Threshold: 0.45


,Product Name,Similar Product Name,Distance,Threshold
0,"Howard Miller 13-3/4"" Diameter Brushed Chrome ...","Howard Miller 14-1/2"" Diameter Chrome Round Wa...",0.402744,0.45
1,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...","Howard Miller 13-3/4"" Diameter Brushed Chrome ...",0.402744,0.45


In [19]:
# FAISS with Sentence Transformer
from sentence_transformers import SentenceTransformer

# Initiate the model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Encode texts into dense embeddings
embeddings = model.encode(df_prods_filtered["Product Name"].tolist(),
                          convert_to_numpy=True,
                          normalize_embeddings=True).astype("float32")


# Build FAISS index (cosine ≈ inner-product if normalized)
index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)

# 3. Search for nearest neighbors
k = 5
D, I = index.search(embeddings, k)

# Example: show neighbors of row 0
for idx, score in zip(I[0], D[0]):
    print(df_prods_filtered.iloc[idx]["Product Name"], "— score:", score)


# Showing with the 90% threshold directly
threshold = 0.90
similar_items = []
for i, (dist, idx) in enumerate(zip(D, I)):
    for d, j in zip(dist[1:], idx[1:]):  # Skip the first one (itself)
        if d > threshold:  # Note: higher is more similar in inner-product
            similar_items.append({
                "Product Name": df_prods_filtered.iloc[i]['Product Name'],
                "Similar Product Name": df_prods_filtered.iloc[j]['Product Name'],
                "Score": d
            })

Eldon Executive Woodline II Desk Accessories, Mahogany — score: 1.0
Eldon 500 Class Desk Accessories — score: 0.75177866
Eldon Image Series Desk Accessories, Burgundy — score: 0.72849596
Eldon 200 Class Desk Accessories, Black — score: 0.68813837
DAX Wood Document Frame — score: 0.44085842


In [20]:
similar_items_df = pd.DataFrame(similar_items)

display(similar_items_df)

# Threshold is higher on this method, but the values indicate completely different products.


,Product Name,Similar Product Name,Score
0,Xerox 1908,Xerox 1916,0.919559
1,Xerox 1908,Xerox 1888,0.917141
2,"Howard Miller 13-3/4"" Diameter Brushed Chrome ...","Howard Miller 14-1/2"" Diameter Chrome Round Wa...",0.959433
3,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...","Howard Miller 13-3/4"" Diameter Brushed Chrome ...",0.959433
4,Xerox 1916,Xerox 1908,0.919559
5,Xerox 1888,Xerox 1908,0.917141


combining all three methods: the conclusion is that all product ID - name pairs are unique products

#### Part 2: Same Product Names Linked with Different Product IDs

In [21]:


# Finding Product Names with more than one Unique Product ID
product_name_id_counts = df.groupby("Product Name")["Product ID"].nunique()
product_names_with_multiple_ids = product_name_id_counts[product_name_id_counts > 1].reset_index()

print("\nProduct Names linked to multiple Product IDs:")
if not product_names_with_multiple_ids.empty:
    # Filter the original dataframe to show the rows with these product names
    display(df[df["Product Name"].isin(product_names_with_multiple_ids["Product Name"])][["Product Name", "Product ID"]].sort_values(by="Product Name").drop_duplicates())
else:
    print("No product names are linked to multiple product IDs.")

# These ones mostly have generic names, but also some are very specific product names.
# However the assumption on this analysis is that if the product IDs are different there's a reason behind it.# These could be variations of the same product.
# No actions


Product Names linked to multiple Product IDs:


,Product Name,Product ID
7213,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes",OFF-EN-10000781
5049,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes",OFF-EN-10000461
1966,Avery Non-Stick Binders,OFF-BI-10004140
9378,Avery Non-Stick Binders,OFF-BI-10000829
4538,Easy-staple paper,OFF-PA-10000249
1644,Easy-staple paper,OFF-PA-10000474
1653,Easy-staple paper,OFF-PA-10000349
1715,Easy-staple paper,OFF-PA-10004947
9509,Easy-staple paper,OFF-PA-10002764
4781,Easy-staple paper,OFF-PA-10003127


#### Conclusion

In [22]:
# Assign Unique Product IDs for each Product ID - Product Name Pair
df["Unique Product ID"] = df.groupby(["Product ID", "Product Name"]).ngroup() # values are not important for this analysis

df.drop(columns="Product ID", axis=1, inplace=True) # drop the old Product ID

In [23]:
df.to_csv("orders_oct27_assigned_unique_productIDs.csv")